In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

import numpy as np
import matplotlib.pyplot as plt
import os
from src import data
import json
from tqdm.auto import tqdm
from src.metrics import AggregateMetric

from src.utils.sweep_utils import read_sweep_results, relation_from_dict
import logging
from src.utils import logging_utils
from src import hparams

# logger = logging.getLogger(__name__)

# logging.basicConfig(
#     level=logging.DEBUG,
#     format = logging_utils.DEFAULT_FORMAT,
#     datefmt=logging_utils.DEFAULT_DATEFMT,
#     stream=sys.stdout
# )

In [3]:
############################################
sweep_root = "../results/sweep-24-trials"
model_name = "gptj"
############################################

sweep_path = f"{sweep_root}/{model_name}"

In [4]:
sweep_results = read_sweep_results(
    sweep_path, 
    # relation_names=["country capital city"]
)
list(sweep_results.keys())

['person occupation',
 'landmark in country',
 'adjective antonym',
 'person mother',
 'country capital city',
 'plays pro sport',
 'person plays instrument',
 'person university',
 'city in country',
 'food from country',
 'company hq',
 'occupation gender',
 'occupation age',
 'name gender',
 'word first letter',
 'country language',
 'object superclass',
 'name religion',
 'person native language',
 'president election year',
 'fruit outside color',
 'superhero archnemesis',
 'work location',
 'landmark on continent',
 'person lead singer of band',
 'task person type',
 'characteristic gender',
 'country largest city',
 'country currency',
 'fruit inside color',
 'task done by tool',
 'verb past tense',
 'star constellation name',
 'pokemon evolution',
 'president birth year',
 'product by company',
 'name birthplace',
 'word last letter',
 'word sentiment',
 'company CEO',
 'superhero person',
 'person father',
 'substance phase of matter',
 'person sport position',
 'adjective sup

In [5]:
relation_dict = {}
for relation_name, sweep_dict in tqdm(sweep_results.items()):
    relation_dict[relation_name] = relation_from_dict(sweep_dict)
    

  0%|          | 0/47 [00:00<?, ?it/s]

In [6]:
for relation_name, relation in relation_dict.items():
    best_hparams = relation.best_by_efficacy(beta=2.25)
    performance = f"efficacy={best_hparams.efficacy.mean:.3f} | faithfulness={best_hparams.recall.mean:.3f}"
    print(f"{relation_name} >> layer={best_hparams.layer} | beta={best_hparams.beta.mean} | rank={int(best_hparams.rank.mean)} <> {performance}")

    hparams.RelationHParams(
        relation_name=relation.relation_name,
        h_layer=best_hparams.layer,
        h_layer_edit=best_hparams.layer,
        z_layer=-1,
        beta=best_hparams.beta.mean,
        rank=int(np.floor(best_hparams.rank.mean)),
        model_name=model_name,
    ).save()

person occupation >> layer=8 | beta=2.25 | rank=131 <> efficacy=0.656 | faithfulness=0.494
landmark in country >> layer=6 | beta=2.25 | rank=97 <> efficacy=0.679 | faithfulness=0.364
adjective antonym >> layer=8 | beta=2.25 | rank=243 <> efficacy=0.859 | faithfulness=0.690
person mother >> layer=6 | beta=2.25 | rank=170 <> efficacy=0.392 | faithfulness=0.142
country capital city >> layer=3 | beta=2.25 | rank=68 <> efficacy=0.990 | faithfulness=0.880
plays pro sport >> layer=6 | beta=2.25 | rank=117 <> efficacy=0.940 | faithfulness=0.760
person plays instrument >> layer=9 | beta=2.25 | rank=198 <> efficacy=0.763 | faithfulness=0.590
person university >> layer=4 | beta=2.25 | rank=153 <> efficacy=0.907 | faithfulness=0.641
city in country >> layer=2 | beta=2.25 | rank=115 <> efficacy=0.894 | faithfulness=0.442
food from country >> layer=3 | beta=2.25 | rank=113 <> efficacy=0.967 | faithfulness=0.514
company hq >> layer=6 | beta=2.25 | rank=126 <> efficacy=0.493 | faithfulness=0.205
occup

In [8]:
hparams.RelationHParams.from_relation(
    model = "gptj",
    relation = "country capital city",
)

RelationHParams(model_name='gptj', relation_name='country capital city', h_layer=3, beta=2.25, rank=68, z_layer=-1, h_layer_edit=3)

In [ ]:
for relation_name, relation in relation_dict.items():
    best_hparams = relation.best_by_faithfulness(beta=2.5)
    performance = f"efficacy={best_hparams.efficacy.mean:.3f} | faithfulness={best_hparams.recall.mean:.3f}"
    print(f"{relation_name} >> layer={best_hparams.layer} | beta={best_hparams.beta.mean} | rank={int(best_hparams.rank.mean)} <> {performance}")

    hparams.RelationHParams(
        relation_name=relation.relation_name,
        h_layer=best_hparams.layer,
        h_layer_edit=best_hparams.layer,
        z_layer=-1,
        beta=best_hparams.beta.mean,
        rank=int(np.floor(best_hparams.rank.mean)),
        model_name=model_name,
    ).save()